# Logistics for Healthcare project

### DATA CLOUD SPACE SET UP: FIREBASE

In [ ]:
# --Importing necessary libraries databsase setup: firebase. 
import firebase_admin
from firebase_admin import credentials, firestore
from datetime import datetime
import logging

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(message)s')
logger = logging.getLogger(__name__)

# Initialize Firebase
cred = credentials.Certificate('healthy-healthcare-firebase-adminsdk-fbsvc-05315e3efc.json')
if not firebase_admin._apps:
    firebase_admin.initialize_app(cred)
db = firestore.client()

# Initial data for facilities
FACILITIES = {
    "Arnhem": {
        "General Practitioner": 1800,
        "Geriatric Rehabilitation": 800,
        "Short-Term Residential Care": 850,
        "Crisis Admission": 750,
        "Clinical Ward": 800,
        "Acute Admission Unit": 700,
        "Emergency Department": 900,
        "Nursing Home": 4000
    },
    "Nijmegen": {
        "General Practitioner": 533,
        "Geriatric Rehabilitation": 400,
        "Short-Term Residential Care": 450,
        "Crisis Admission": 500,
        "Clinical Ward": 450,
        "Acute Admission Unit": 400,
        "Emergency Department": 600,
        "Nursing Home": 3000
    }
}

def initialize_cloud_space():
    base_date = datetime(2025, 5, 1)
    week_date = base_date.strftime('%Y-%m-%d')
    
    for facility in FACILITIES:
        facility_ref = db.collection('prediction').document(facility)
        facility_ref.set({
            'name': facility,
            'capacities': FACILITIES[facility]
        })
        
        week_ref = facility_ref.collection('weeks').document(f'week_{week_date}')
        week_ref.set({
            'date': week_date,
            'active_patients_data': [],
            'metrics': {},
            'timestamp': firestore.SERVER_TIMESTAMP
        })
        
        logger.info(f"Initialized {facility} for week {week_date}")

if __name__ == "__main__":
    initialize_cloud_space()
    logger.info("Cloud space setup complete")

### SIMULATION MODEL INTERGRATE WITH FIREBASE

In [ ]:
# --Importing necessary libraries for simulation--

import simpy
import numpy as np
from collections import defaultdict
import firebase_admin
from firebase_admin import credentials, firestore
from datetime import datetime, timedelta
import logging
from scipy.stats import expon

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(message)s')
logger = logging.getLogger(__name__)
VERBOSE_LOGGING = True #VERBOSE_LOGGING enables detailed debug logs (e.g., state transitions, capacity usage).

# Initialize Firebase
cred = credentials.Certificate('healthy-healthcare-firebase-adminsdk-fbsvc-05315e3efc.json') #Loads Firebase credentials from JSON file
if not firebase_admin._apps:
    firebase_admin.initialize_app(cred)
db = firestore.client()

# Facility capacities for simulation
# These capacities are used to simulate the healthcare system's resource limits each region.
# They are also used to initialize the Firebase database with the initial state of each facility.
FACILITIES = {
    "Arnhem": {
        "General Practitioner": 1800,
        "Geriatric Rehabilitation": 800,
        "Short-Term Residential Care": 850,
        "Crisis Admission": 750,
        "Clinical Ward": 800,
        "Acute Admission Unit": 700,
        "Emergency Department": 900,
        "Nursing Home": 4000
    },
    "Nijmegen": {
        "General Practitioner": 533,
        "Geriatric Rehabilitation": 400,
        "Short-Term Residential Care": 450,
        "Crisis Admission": 500,
        "Clinical Ward": 450,
        "Acute Admission Unit": 400,
        "Emergency Department": 600,
        "Nursing Home": 3000
    }
}

# Valid states for simulation
# These states represent the different stages a patient can be in during their healthcare journey.
# Discharge and Passed Away are terminal; others are active stages.
# Awaiting Nursing Home (W2) represents the wait for Nursing Home (W2 period, mean 96 hours).
# W1 is not a state but a wait within Home (mean 48 hours).
VALID_STATES = {
    "Home",
    "General Practitioner",
    "Geriatric Rehabilitation",
    "Short-Term Residential Care",
    "Crisis Admission",
    "Awaiting Nursing Home (W2)",
    "Clinical Ward",
    "Acute Admission Unit",
    "Emergency Department",
    "Nursing Home",
    "Discharge",
    "Passed Away"
}

# Simulation Parameters
TOTAL_SIMULATION_WEEKS = 56
NUM_NEW_PATIENTS_PER_WEEK = {"Arnhem": 300, "Nijmegen": 250}  # Fixed new patients per week for each region. This + the count of active patients (fetch from Firebase) determines the total number of patients in the simulation.
SIM_TIME_PER_WEEK = 28 * 24
W1_MEAN = 48
W2_MEAN = 96

# Probabilities and durations for various healthcare processes
# These probabilities and durations are used to simulate patient transitions between states.
PASS_AWAY_PROB = 0.02
TRANSFER_PROB = 0.3
EMERGENCY_PROB = 0.4
ED_PROB_DISCHARGE_HOME = 0.1
ED_PROB_TO_STC = 0.25

# Stay durations in hours for various states
NURSING_HOME_STAY_MIN_HOURS = 6 * 30 * 24
NURSING_HOME_STAY_MAX_HOURS = 3 * 12 * 30 * 24
ED_STC_TARGET_OPTIONS = ["Crisis Admission", "Short-Term Residential Care", "Geriatric Rehabilitation"]
ED_STC_ADMISSION_WAIT_MIN_HOURS = 2
ED_STC_ADMISSION_WAIT_MAX_HOURS = 12
GP_CONSULT_MIN_HOURS = 0.25
GP_CONSULT_MAX_HOURS = 1
STC_STAY_MIN_HOURS = 72
STC_STAY_MAX_HOURS = 120
STC_TRANSFER_MIN_HOURS = 6
STC_TRANSFER_MAX_HOURS = 12
CW_STAY_MIN_HOURS = 48
CW_STAY_MAX_HOURS = 120
AAU_STAY_MIN_HOURS = 48
AAU_STAY_MAX_HOURS = 120
ED_STAY_MIN_HOURS = 24
ED_STAY_MAX_HOURS = 48
CW_ADMISSION_WAIT_MIN_HOURS = 2  
CW_ADMISSION_WAIT_MAX_HOURS = 12
AAU_ADMISSION_WAIT_MIN_HOURS = 2  
AAU_ADMISSION_WAIT_MAX_HOURS = 12
ED_ADMISSION_WAIT_MIN_HOURS = 2  
ED_ADMISSION_WAIT_MAX_HOURS = 12

# Stats Class
# The Stats class tracks simulation metrics, including occupancy, wait times, and new exit predictions.
class Stats:
    def __init__(self, env, resources, week_number, facility, capacities, active_patients_counts):
        self.env = env # SimPy environment.
        self.resources = resources # SimPy resources (beds per stage).
        self.week_number = week_number # Current week (e.g., 1 for 2025-05-01)
        self.facility = facility # Facility name (each region) (e.g., "Arnhem" or "Nijmegen").
        self.capacities = capacities # Bed capacities (from prediction or FACILITIES).
        self.active_patients_counts = active_patients_counts # Counts from real-time data (e.g., {"Nursing Home": 60}).

        # Attributes to track simulation metrics
        self.state_visits = defaultdict(int) # Counts visits to each stage.
        self.state_times = defaultdict(list) # Lists durations spent in each stage (within week)
        self.wait_elv_times = [] # Wait times for Geriatric Rehabilitation, Short-Term Residential Care.
        self.wait_crisis_times = [] # Wait times for Crisis Admission.
        self.wait_cw_times = [] # Wait times for Clinical Ward. 
        self.wait_aau_times = [] # Wait times for Acute Admission Unit.
        self.wait_ed_times = [] # Wait times for Emergency Department.
        self.wait_w2_times = [] # Wait times for Awaiting Nursing Home (W2).
        self.capacity_usage = defaultdict(float) # Tracks bed usage for occupancy metrics.
        self.capacity_counts = defaultdict(int) # Tracks bed usage for occupancy metrics.

# Do we need to track wait times for other states like Clinical Ward, Acute Admission Unit, Emergency Department?

        # Expected durations (in hours) and exit probabilities
        self.expected_durations = {                                       # Average stay durations per stage (in hours)
            "Home": W1_MEAN,  # Exponential
            "General Practitioner": np.mean([GP_CONSULT_MIN_HOURS, GP_CONSULT_MAX_HOURS]),  # Uniform
            "Geriatric Rehabilitation": np.mean([STC_STAY_MIN_HOURS, STC_STAY_MAX_HOURS]),
            "Short-Term Residential Care": np.mean([STC_STAY_MIN_HOURS, STC_STAY_MAX_HOURS]),
            "Crisis Admission": np.mean([STC_STAY_MIN_HOURS, STC_STAY_MAX_HOURS]),
            "Awaiting Nursing Home (W2)": W2_MEAN,  # Exponential
            "Clinical Ward": np.mean([CW_STAY_MIN_HOURS, CW_STAY_MAX_HOURS]),
            "Acute Admission Unit": np.mean([AAU_STAY_MIN_HOURS, AAU_STAY_MAX_HOURS]),
            "Emergency Department": np.mean([ED_STAY_MIN_HOURS, ED_STAY_MAX_HOURS]),
            "Nursing Home": np.mean([NURSING_HOME_STAY_MIN_HOURS, NURSING_HOME_STAY_MAX_HOURS])
        }
        self.exit_probabilities = self.calculate_exit_probabilities() 
        self.env.process(self.monitor_capacity())
# Calculate Exit Probabilities (a solution to tackle the issue of patients tracking duration stay of each patients that are updated from Firebase)
    def calculate_exit_probabilities(self): # P(stay ≤ 168 hours) per stage
        """Calculate probability of exiting each stage within one week (168 hours)."""
        week_hours = SIM_TIME_PER_WEEK
        probs = {}
        for state in VALID_STATES - {"Discharge", "Passed Away"}:
            if state == "Home":
                probs[state] = 1 - expon.cdf(week_hours, scale=W1_MEAN)
            elif state == "General Practitioner":
                min_h, max_h = GP_CONSULT_MIN_HOURS, GP_CONSULT_MAX_HOURS
                probs[state] = (week_hours - min_h) / (max_h - min_h) if min_h <= week_hours <= max_h else 1 if week_hours > max_h else 0
            elif state in ["Geriatric Rehabilitation", "Short-Term Residential Care", "Crisis Admission"]:
                min_h, max_h = STC_STAY_MIN_HOURS, STC_STAY_MAX_HOURS
                probs[state] = (week_hours - min_h) / (max_h - min_h) if min_h <= week_hours <= max_h else 1 if week_hours > max_h else 0
            elif state == "Awaiting Nursing Home (W2)":
                probs[state] = 1 - expon.cdf(week_hours, scale=W2_MEAN)
            elif state == "Clinical Ward":
                min_h, max_h = CW_STAY_MIN_HOURS, CW_STAY_MAX_HOURS
                probs[state] = (week_hours - min_h) / (max_h - min_h) if min_h <= week_hours <= max_h else 1 if week_hours > max_h else 0
            elif state == "Acute Admission Unit":
                min_h, max_h = AAU_STAY_MIN_HOURS, AAU_STAY_MAX_HOURS
                probs[state] = (week_hours - min_h) / (max_h - min_h) if min_h <= week_hours <= max_h else 1 if week_hours > max_h else 0
            elif state == "Emergency Department":
                min_h, max_h = ED_STAY_MIN_HOURS, ED_STAY_MAX_HOURS
                probs[state] = (week_hours - min_h) / (max_h - min_h) if min_h <= week_hours <= max_h else 1 if week_hours > max_h else 0
            elif state == "Nursing Home":
                min_h, max_h = NURSING_HOME_STAY_MIN_HOURS, NURSING_HOME_STAY_MAX_HOURS
                probs[state] = (week_hours - min_h) / (max_h - min_h) if min_h <= week_hours <= max_h else 1 if week_hours > max_h else 0
        return probs

# Record Visits, Time, and Transitions
    # These methods are used to log visits to states, time spent in states, and transitions between states.
    def record_visit(self, state):
        self.state_visits[state] += 1
        if VERBOSE_LOGGING:
            logger.debug(f"{self.facility} Week {self.week_number}: Visit to {state}")

    def record_time(self, state, duration):
        if duration > 0.001:
            self.state_times[state].append(duration)
            if VERBOSE_LOGGING:
                logger.debug(f"{self.facility} Week {self.week_number}: Time in {state}: {duration:.2f} hours")

    def record_transition(self, from_state, to_state, duration):
        if to_state in ["Geriatric Rehabilitation", "Short-Term Residential Care"]:
            self.wait_elv_times.append(duration)
            if VERBOSE_LOGGING:
                logger.debug(f"{self.facility} Week {self.week_number}: ELV wait time: {duration:.2f} hours")
        elif to_state == "Crisis Admission":
            self.wait_crisis_times.append(duration)
            if VERBOSE_LOGGING:
                logger.debug(f"{self.facility} Week {self.week_number}: Crisis wait time: {duration:.2f} hours")
        elif to_state == "Clinical Ward":
            self.wait_cw_times.append(duration)
            if VERBOSE_LOGGING:
                logger.debug(f"{self.facility} Week {self.week_number}: Clinical Ward wait time: {duration:.2f} hours")
        elif to_state == "Acute Admission Unit":
            self.wait_aau_times.append(duration)
            if VERBOSE_LOGGING:
                logger.debug(f"{self.facility} Week {self.week_number}: Acute Admission wait time: {duration:.2f} hours")
        elif to_state == "Emergency Department":
            self.wait_ed_times.append(duration)
            if VERBOSE_LOGGING:
                logger.debug(f"{self.facility} Week {self.week_number}: Emergency Department wait time: {duration:.2f} hours")
        elif to_state == "Nursing Home":  
            self.wait_w2_times.append(duration)
            if VERBOSE_LOGGING:
                logger.debug(f"{self.facility} Week {self.week_number}: W2 wait time: {duration:.2f} hours")

# Monitor Capacity Usage    
    # This method monitors the usage of resources (beds) in each state every 12 hours.
    # Used for occupancy metrics (e.g., OCC_NH).
    def monitor_capacity(self):
        while True:
            for state_name, resource_obj in self.resources.items():
                if state_name in self.capacities:
                    usage = len(resource_obj.users)
                    self.capacity_usage[state_name] += usage
                    self.capacity_counts[state_name] += 1
                    if VERBOSE_LOGGING and usage > 0:
                        logger.debug(f"{self.facility} Week {self.week_number}: {state_name} usage: {usage}/{self.capacities[state_name]}")
            yield self.env.timeout(12)

# Get Metrics 
    # This method calculates and returns various metrics for the week, including occupancy rates, wait times, expected durations, and expected exits.
    # It returns a dictionary of metrics that can be stored in Firebase.
    def get_metrics(self, week_start_date):
        # Beds: beds/available "slots" for each state.
        elv_beds = self.capacities["Geriatric Rehabilitation"] + self.capacities["Short-Term Residential Care"]
        crisis_beds = self.capacities["Crisis Admission"]
        nh_beds = self.capacities["Nursing Home"]
        cw_beds = self.capacities["Clinical Ward"]
        aau_beds = self.capacities["Acute Admission Unit"]
        ed_beds = self.capacities["Emergency Department"]
        # Occupancy: Average usage over counts, divided by capacity
        occ_elv = ((self.capacity_usage["Geriatric Rehabilitation"] + self.capacity_usage["Short-Term Residential Care"]) /
                   (self.capacity_counts["Geriatric Rehabilitation"] + self.capacity_counts["Short-Term Residential Care"])) / elv_beds if (self.capacity_counts["Geriatric Rehabilitation"] + self.capacity_counts["Short-Term Residential Care"]) > 0 else 0
        occ_nh = (self.capacity_usage["Nursing Home"] / self.capacity_counts["Nursing Home"]) / nh_beds if self.capacity_counts["Nursing Home"] > 0 else 0
        occ_cw = (self.capacity_usage["Clinical Ward"] / self.capacity_counts["Clinical Ward"]) / cw_beds if self.capacity_counts["Clinical Ward"] > 0 else 0
        occ_aau = (self.capacity_usage["Acute Admission Unit"] / self.capacity_counts["Acute Admission Unit"]) / aau_beds if self.capacity_counts["Acute Admission Unit"] > 0 else 0
        occ_ed = (self.capacity_usage["Emergency Department"] / self.capacity_counts["Emergency Department"]) / ed_beds if self.capacity_counts["Emergency Department"] > 0 else 0
        occ_crisis = (self.capacity_usage["Crisis Admission"] / self.capacity_counts["Crisis Admission"]) / crisis_beds if self.capacity_counts["Crisis Admission"] > 0 else 0
        # Wait times: Mean of wait_elv_times/wait_crisis_times in days, defaults to W2_MEAN or average wait.
        wait_elv = np.mean(self.wait_elv_times) / 24 if self.wait_elv_times else W2_MEAN / 24
        wait_crisis = np.mean(self.wait_crisis_times) / 24 if self.wait_crisis_times else np.mean([ED_STC_ADMISSION_WAIT_MIN_HOURS, ED_STC_ADMISSION_WAIT_MAX_HOURS]) / 24
        wait_cw = np.mean(self.wait_cw_times) / 24 if self.wait_cw_times else np.mean([CW_ADMISSION_WAIT_MIN_HOURS, CW_ADMISSION_WAIT_MAX_HOURS]) / 24
        wait_aau = np.mean(self.wait_aau_times) / 24 if self.wait_aau_times else np.mean([AAU_ADMISSION_WAIT_MIN_HOURS, AAU_ADMISSION_WAIT_MAX_HOURS]) / 24
        wait_ed = np.mean(self.wait_ed_times) / 24 if self.wait_ed_times else np.mean([ED_ADMISSION_WAIT_MIN_HOURS, ED_ADMISSION_WAIT_MAX_HOURS]) / 24
        wait_w2 = np.mean(self.wait_w2_times) / 24 if self.wait_w2_times else W2_MEAN / 24
        # Expected durations (in days)
        expected_durations = {f"EXPECTED_DURATION_{state.upper().replace(' ', '_')}": duration / 24 for state, duration in self.expected_durations.items()}

        # Expected exits based on active_patients_counts
        expected_exits = {}
        for state in VALID_STATES - {"Discharge", "Passed Away"}:
            count = self.active_patients_counts.get(state, 0)
            prob = self.exit_probabilities.get(state, 0)
            expected_exits[f"EXPECTED_EXITS_{state.upper().replace(' ', '_')}"] = count * prob

        metrics = {
            "MUNICIPALITY": self.facility,
            "DATE": week_start_date,
            "ELVBEDS": elv_beds,
            "CRISISBEDS": crisis_beds,
            "NHBEDS": nh_beds,
            "Clinical Ward Beds": cw_beds,
            "Acute Admission Unit Beds": aau_beds,
            "Emergency Department Beds": ed_beds,
            "WAIT_ELV": float(wait_elv),
            "WAIT_CRISIS": float(wait_crisis),
            "WAIT_CW": float(wait_cw),
            "WAIT_AAU": float(wait_aau),
            "WAIT_ED": float(wait_ed),
            "WAIT_W2": float(wait_w2),
            "OCC_ELV": float(occ_elv),
            "OCC_CRISIS": float(occ_crisis),
            "OCC_NH": float(occ_nh),
            "OCC_CW": float(occ_cw),
            "OCC_AAU": float(occ_aau),
            "OCC_ED": float(occ_ed),
            **expected_durations,
            **expected_exits
        }
        if VERBOSE_LOGGING:
            logger.debug(f"{self.facility} Week {self.week_number}: Capacities - NH: {nh_beds}, ELV: {elv_beds}, Crisis: {crisis_beds}, CW: {cw_beds}, AAU: {aau_beds}, ED: {ed_beds}")
            logger.debug(f"{self.facility} Week {self.week_number}: Metrics: {metrics}")
        return metrics

# Client Class
# The Client class represents a patient in the healthcare system.
class Client:
    def __init__(self, env, client_id, stats, resources, initial_state="Home"):
        self.env = env
        self.id = client_id
        self.state = initial_state
        self.alive = True
        self.stats = stats
        self.state_entry_time = env.now
        self.resources = resources
        self.state_handlers = {
            "Home": self.at_home,
            "General Practitioner": self.at_gp,
            "Geriatric Rehabilitation": self.in_short_term_care,
            "Short-Term Residential Care": self.in_short_term_care,
            "Crisis Admission": self.in_short_term_care,
            "Awaiting Nursing Home (W2)": self.await_nursing_home_w2,
            "Clinical Ward": self.in_clinical_ward,
            "Acute Admission Unit": self.in_acute_admission,
            "Emergency Department": self.in_emergency_dept,
            "Nursing Home": self.in_nursing_home
        }
        self.stats.record_visit(self.state)
        # Initializes a patient with an ID, starting state (default Home), and tracks entry time.
        # state_handlers maps stages to methods (e.g., Home → at_home).

    def run(self):
        while self.alive and self.state not in ["Discharge", "Passed Away"]:
            if self.env.now >= SIM_TIME_PER_WEEK:
                break
            handler = self.state_handlers.get(self.state)
            if handler:
                if self.state in self.resources:
                    with self.resources[self.state].request() as req:
                        yield req
                        yield self.env.process(handler())
                else:
                    yield self.env.process(handler())
            else:
                self.alive = False
                break
    # Runs until the patient dies, is discharged, or the week ends (168 hours).
    # Requests resources (beds) for stages with capacity limits.
    # Executes the stage’s handler method.

    def update_state(self, new_state): # Updates state, records duration, and logs transitions.
        if not self.alive and self.state == "Passed Away" and new_state == "Passed Away": return
        if not self.alive and new_state != "Passed Away": return
        duration = self.env.now - self.state_entry_time
        self.stats.record_time(self.state, duration)
        self.stats.record_transition(self.state, new_state, duration)
        self.state = new_state
        self.state_entry_time = self.env.now
        if self.state == "Passed Away":
            self.alive = False
        else:
            self.stats.record_visit(self.state)
        if VERBOSE_LOGGING:
            logger.debug(f"Client {self.id} in {self.stats.facility}: Transition to {new_state}")

    def _check_pass_away(self, probability, context_state): # Checks if the client passes away based on a probability.
        if self.alive and np.random.random() < probability:
            self.update_state("Passed Away")
            return True
        return False

# Below is how  the Client class methods are structured to handle different states in the healthcare journey.
# Each method simulates the time spent in that state and transitions to the next state based on probabilities and conditions.
# Goes untill # Simulation runner, for the sake of simplicity, we only explain 2 states here as an example, the rest follow a similar pattern.
    def at_home(self): # Either goes to Emergency Department (EMERGENCY_PROB = 0.4) or waits W1 (exponential(48)) then to General Practitioner
        if not self.alive: return
        if np.random.random() < EMERGENCY_PROB:
            ed_wait = np.random.uniform(ED_ADMISSION_WAIT_MIN_HOURS, ED_ADMISSION_WAIT_MAX_HOURS)
            yield self.env.timeout(ed_wait)
            self.update_state("Emergency Department")
        else:
            w1_wait_time = np.random.exponential(W1_MEAN)
            yield self.env.timeout(w1_wait_time)
            if not self.alive: return
            if self._check_pass_away(PASS_AWAY_PROB, "at W1 (waiting for GP)"): return
            self.update_state("General Practitioner")

    def at_gp(self): # Stays for (0.25, 1) hours (stay uniform), then to Short term care (STC) stage (40% Geriatric Rehabilitation, 40% Short-Term Residential Care, 20% Crisis Admission) after wait  (uniform((0, 48)).
        if not self.alive: return
        gp_consult_time = np.random.uniform(GP_CONSULT_MIN_HOURS, GP_CONSULT_MAX_HOURS)
        yield self.env.timeout(gp_consult_time)
        if not self.alive: return
        rand_stc = np.random.random()
        next_stc_type = ("Geriatric Rehabilitation" if rand_stc < 0.4 else
                         "Short-Term Residential Care" if rand_stc < 0.8 else
                         "Crisis Admission")
        stc_admission_wait = np.random.uniform(0, 48)
        yield self.env.timeout(stc_admission_wait)
        if not self.alive: return
        self.update_state(next_stc_type)

    def in_short_term_care(self): 
        if not self.alive: return
        current_stc_type = self.state
        stay_time = np.random.uniform(STC_STAY_MIN_HOURS, STC_STAY_MAX_HOURS)
        yield self.env.timeout(stay_time)
        if not self.alive: return
        if np.random.random() < TRANSFER_PROB:
            options = ["Geriatric Rehabilitation", "Short-Term Residential Care", "Crisis Admission"]
            if current_stc_type in options: options.remove(current_stc_type)
            if options:
                new_stc_type = np.random.choice(options)
                transfer_delay = np.random.uniform(STC_TRANSFER_MIN_HOURS, STC_TRANSFER_MAX_HOURS)
                yield self.env.timeout(transfer_delay)
                if not self.alive: return
                self.update_state(new_stc_type)
                return
        rand_next = np.random.random()
        if rand_next < 0.4:
            cw_wait = np.random.uniform(CW_ADMISSION_WAIT_MIN_HOURS, CW_ADMISSION_WAIT_MAX_HOURS)  # New: Wait before CW
            yield self.env.timeout(cw_wait)
            next_state = "Clinical Ward"
        elif rand_next < 0.7:
            aau_wait = np.random.uniform(AAU_ADMISSION_WAIT_MIN_HOURS, AAU_ADMISSION_WAIT_MAX_HOURS)  # New: Wait before AAU
            yield self.env.timeout(aau_wait)
            next_state = "Acute Admission Unit"
        else:
            ed_wait = np.random.uniform(ED_ADMISSION_WAIT_MIN_HOURS, ED_ADMISSION_WAIT_MAX_HOURS)  # New: Wait before ED
            yield self.env.timeout(ed_wait)
            next_state = "Emergency Department"
        self.update_state(next_state)

    def _handle_post_acute_care_transition(self):
        if not self.alive: return
        self.update_state("Awaiting Nursing Home (W2)")

    def in_clinical_ward(self):
        if not self.alive: return
        stay_time = np.random.uniform(CW_STAY_MIN_HOURS, CW_STAY_MAX_HOURS)
        yield self.env.timeout(stay_time)
        if not self.alive: return
        self._handle_post_acute_care_transition()

    def in_acute_admission(self):
        if not self.alive: return
        stay_time = np.random.uniform(AAU_STAY_MIN_HOURS, AAU_STAY_MAX_HOURS)
        yield self.env.timeout(stay_time)
        if not self.alive: return
        self._handle_post_acute_care_transition()

    def in_emergency_dept(self):
        if not self.alive: return
        stay_time = np.random.uniform(ED_STAY_MIN_HOURS, ED_STAY_MAX_HOURS)
        yield self.env.timeout(stay_time)
        if not self.alive: return
        rand_ed_outcome = np.random.random()
        if rand_ed_outcome < ED_PROB_DISCHARGE_HOME:
            self.update_state("Discharge")
        elif rand_ed_outcome < ED_PROB_DISCHARGE_HOME + ED_PROB_TO_STC:
            if not ED_STC_TARGET_OPTIONS:
                self._handle_post_acute_care_transition()
                return
            chosen_stc_from_ed = np.random.choice(ED_STC_TARGET_OPTIONS)
            stc_admission_wait_from_ed = np.random.uniform(ED_STC_ADMISSION_WAIT_MIN_HOURS, ED_STC_ADMISSION_WAIT_MAX_HOURS)
            yield self.env.timeout(stc_admission_wait_from_ed)
            if not self.alive: return
            self.update_state(chosen_stc_from_ed)
        else:
            self._handle_post_acute_care_transition()

    def await_nursing_home_w2(self):
        if not self.alive: return
        w2_wait_time = np.random.exponential(W2_MEAN)
        yield self.env.timeout(w2_wait_time)
        if not self.alive: return
        self.update_state("Nursing Home")

    def in_nursing_home(self):
        if not self.alive: return
        stay_duration = np.random.uniform(NURSING_HOME_STAY_MIN_HOURS, NURSING_HOME_STAY_MAX_HOURS)
        yield self.env.timeout(stay_duration)
        if not self.alive: return
        self.update_state("Discharge")

# Simulation Runner
def run_one_week_simulation(week_num, active_patients_counts, global_client_id_counter_ref, facility, week_start_date, capacities, new_patients):
    try:
        validated_capacities = {}
        for state, cap in capacities.items():
            expected_cap = FACILITIES[facility].get(state, 1)
            if not isinstance(cap, int) or cap <= 0 or abs(cap - expected_cap) > 0.01 * expected_cap:
                logger.warning(f"Invalid or inconsistent capacity for {state} in {facility}: {cap}. Using {expected_cap}.")
                cap = expected_cap
            validated_capacities[state] = cap
        
        env = simpy.Environment()
        resources = {state: simpy.Resource(env, capacity=cap) for state, cap in validated_capacities.items()}
        stats = Stats(env, resources, week_num, facility, validated_capacities, active_patients_counts)
        current_week_clients = []
        
        # Initialize clients from active_patients_counts
        initial_total = sum(active_patients_counts.values())
        logger.info(f"{facility} Week {week_num}: Initial active patients: {initial_total}")

        for state, count in active_patients_counts.items(): # active_patients_counts: the counts of patients fetched from Firebase for the current week.
            if state not in VALID_STATES or state in ["Discharge", "Passed Away"]:
                if state == "Awaiting Nursing Home":
                    logger.warning(f"Invalid or terminal state {state} in {facility} active_patients_counts: {count}. Did you mean 'Awaiting Nursing Home (W2)'?")
                else:
                    logger.warning(f"Invalid or terminal state {state} in {facility} active_patients_counts: {count}")
                continue
            if not isinstance(count, int) or count < 0:
                logger.warning(f"Invalid count for {state} in {facility}: {count}. Skipping.")
                continue
            for _ in range(count):
                global_client_id_counter_ref[0] += 1
                client_id = global_client_id_counter_ref[0]
                client = Client(env, client_id, stats, resources, initial_state=state)
                current_week_clients.append(client)
                env.process(client.run())
        
        # Add new patients
        for _ in range(new_patients): # Fixed count of new patients per week.
            global_client_id_counter_ref[0] += 1
            client_id = global_client_id_counter_ref[0]
            client = Client(env, client_id, stats, resources, initial_state="Home")
            current_week_clients.append(client)
            env.process(client.run())
        
        env.run(until=SIM_TIME_PER_WEEK)
        active_patients_data_out = []
        for client in current_week_clients:
            if client.alive and client.state not in ["Discharge", "Passed Away"]:
                active_patients_data_out.append({'id': client.id, 'state': client.state})
                stats.record_time(client.state, env.now - client.state_entry_time)
        logger.info(f"{facility} Week {week_num}: Active patients at end: {len(active_patients_data_out)}, Total processed: {len(current_week_clients)}")
        return active_patients_data_out, global_client_id_counter_ref, stats.get_metrics(week_start_date)
    except Exception as e:
        logger.error(f"Error in simulation for {facility} week {week_num}: {str(e)}")
        raise

    # Steps:
# Validates capacities against FACILITIES.
# Initializes SimPy environment and resources.
# Creates Stats with active_patients_counts.
# Initializes clients from active_patients_counts and new_patients.
# Runs simulation for 168 hours.
#Collects active patients (active_patients_data_out) and metrics.
    # Validation:
# Skips invalid states (e.g., Awaiting Nursing Home → suggests W2).
# Ensures counts are non-negative integers.

# Cloud Interaction
# Fetches active patients counts from Firestore for the given week.
def get_active_patients_counts(facility, week_date):
    try:
        week_ref = db.collection('real-time data').document(facility).collection('weeks').document(f'week_{week_date}')
        doc = week_ref.get()
        if doc.exists:
            active_patients = doc.to_dict().get('active_patients', {})
            logger.info(f"Fetched active patients for {facility} week {week_date}: {active_patients}")
            return active_patients
        else:
            logger.warning(f"No real-time data for {facility} week {week_date}. Using empty active patients.")
            return {}
    except Exception as e:
        logger.error(f"Error fetching active patients for {facility} week {week_date}: {str(e)}")
        return {}
        
def get_capacities(facility):
    try:
        facility_ref = db.collection('prediction').document(facility)
        capacities = facility_ref.get().to_dict().get('capacities', FACILITIES[facility])
        if VERBOSE_LOGGING:
            logger.debug(f"Fetched capacities for {facility}: {capacities}")
        return capacities
    except Exception as e:
        logger.error(f"Error fetching capacities for {facility}: {str(e)}")
        return FACILITIES[facility]

def run_simulation_with_cloud(week_to_run=None):
    facilities = ["Arnhem", "Nijmegen"]
    base_date = datetime(2025, 5, 1)
    
    if week_to_run is not None:
        weeks = [week_to_run]
    else:
        weeks = range(1, TOTAL_SIMULATION_WEEKS + 1)
    
    for week in weeks:
        week_start_date = base_date + timedelta(weeks=week-1)
        week_start_date_str = week_start_date.strftime('%Y-%m-%d')
        next_week_date = week_start_date + timedelta(weeks=1)
        next_week_date_str = next_week_date.strftime('%Y-%m-%d')
        
        for facility in facilities:
            active_patients_counts = get_active_patients_counts(facility, week_start_date_str)
            capacities = get_capacities(facility)
            if not capacities:
                logger.error(f"No capacities found for {facility}. Skipping simulation.")
                continue
            new_patients = NUM_NEW_PATIENTS_PER_WEEK[facility]
            global_client_id_counter = [0]  # Reset counter per week
            
            try:
                np.random.seed(week * facilities.index(facility) * 42 + np.random.randint(1000))
                active_patients_data_out, global_client_id_counter, metrics = run_one_week_simulation(
                    week_num=week,
                    active_patients_counts=active_patients_counts,
                    global_client_id_counter_ref=global_client_id_counter,
                    facility=facility,
                    week_start_date=next_week_date_str,
                    capacities=capacities,
                    new_patients=new_patients
                )
                
                facility_ref = db.collection('prediction').document(facility)
                week_ref = facility_ref.collection('weeks').document(f'week_{next_week_date_str}')
                try:
                    week_ref.set({
                        'date': next_week_date_str,
                        'active_patients_data': active_patients_data_out,
                        'metrics': metrics,
                        'timestamp': firestore.SERVER_TIMESTAMP
                    })
                    logger.info(f"Updated {facility} for week {next_week_date_str} with {new_patients} new patients")
                except Exception as e:
                    logger.error(f"Failed to update Firestore for {facility} week {next_week_date_str}: {str(e)}")
            except Exception as e:
                logger.error(f"Simulation failed for {facility} week {next_week_date_str}: {str(e)}")

if __name__ == "__main__":
    run_simulation_with_cloud(week_to_run=1) # Runs simulation for week 1. If future weeks are needed, change week_to_run to a specific number (starting from 2025-05-01).
    logger.info("Simulation complete with cloud integration")

### BACKUP: LOGIC OF SIMULATION MODEL 

This is for readers to read the output of the script below and understand how a patient go within the simulation model. This script solely serves the logic of the simulation model based on the diagram. This script is not complete, please do not use this one, use the script above íntead

In [30]:
import simpy
import random
import statistics
from collections import defaultdict

# Parameters
SIM_TIME = 1000
NUM_CLIENTS = 100
W1_MEAN = 24
W2_MEAN = 48
PASS_AWAY_PROB = 0.05
TRANSFER_PROB = 0.2
EMERGENCY_PROB = 0.4
NURSING_HOME_STAY_MIN_HOURS = 7 * 24
NURSING_HOME_STAY_MAX_HOURS = 30 * 24

CAPACITY = {
    "General Practitioner": 10,
    "Geriatric Rehabilitation": 10,
    "Short-Term Residential Care": 10,
    "Crisis Admission": 10,
    "Clinical Ward": 10,
    "Acute Admission Unit": 10,
    "Emergency Department": 15,
    "Nursing Home": 20
}

class Stats:
    def __init__(self, env, resources):
        self.env = env
        self.resources = resources
        self.capacity_usage = defaultdict(list)
        self.state_visits = defaultdict(int)
        self.state_times = defaultdict(list)
        self.final_states = defaultdict(int)
        self.pass_away_count = 0
        self.transitions = defaultdict(lambda: defaultdict(int))
        self.env.process(self.monitor_capacity())

    def record_visit(self, state):
        self.state_visits[state] += 1

    def record_time(self, state, duration):
        if duration > 0.001:
            self.state_times[state].append(duration)

    def record_final_state(self, state):
        self.final_states[state] += 1

    def record_pass_away(self):
        self.pass_away_count += 1

    def record_transition(self, from_state, to_state):
        self.transitions[from_state][to_state] += 1

    def monitor_capacity(self):
        while True:
            for state_name, resource_obj in self.resources.items():
                if state_name in CAPACITY:
                    self.capacity_usage[state_name].append(len(resource_obj.users))
            yield self.env.timeout(1)

    def print_summary(self):
        print("\n=== Simulation Statistics Summary ===")
        print("\nNumber of visits to each state:")
        for state, count in self.state_visits.items():
            print(f"{state}: {count} visits")

        print("\nAverage time spent in each state (for clients who completed their stay there):")
        for state, times in sorted(self.state_times.items()):
            if times:
                avg_time = statistics.mean(times)
                print(f"{state}: {avg_time:.2f} hours (based on {len(times)} entries)")
            else:
                print(f"{state}: No time recorded or no clients completed stay")

        print("\nAverage capacity usage (occupancy) per state:")
        for state, usage_data in sorted(self.capacity_usage.items()):
            if state in CAPACITY:
                if usage_data:
                    avg_usage = statistics.mean(usage_data)
                    max_usage = max(usage_data)
                    print(f"{state}: Avg {avg_usage:.2f} clients (Max: {max_usage}, Capacity: {CAPACITY[state]})")
                else:
                    print(f"{state}: No usage recorded (Capacity: {CAPACITY[state]})")

        print("\nTransition counts between states:")
        for from_state, to_states_counts in sorted(self.transitions.items()):
            for to_state, count in sorted(to_states_counts.items()):
                print(f"{from_state} -> {to_state}: {count} transitions")

        print("\nFinal states of clients at SIM_TIME or upon exit:")
        for state, count in sorted(self.final_states.items()):
            print(f"{state}: {count} clients")

        print(f"\nTotal clients who passed away: {self.pass_away_count}")
        if NUM_CLIENTS > 0:
            print(f"Percentage of clients who passed away: {(self.pass_away_count / NUM_CLIENTS * 100):.2f}%")
        else:
            print("Percentage of clients who passed away: N/A (0 clients simulated)")

class Client:
    def __init__(self, env, id, stats, resources):
        self.env = env
        self.id = id
        self.state = "Home"
        self.alive = True
        self.stats = stats
        self.state_entry_time = env.now
        self.resources = resources
        print(f"Client {self.id} starts at Home at time {self.env.now:.2f}") # This print should appear
        self.stats.record_visit(self.state)

    def run(self):
        while self.alive and self.state not in ["Discharge", "Passed Away"]:
            if self.env.now >= SIM_TIME:
                # print(f"Client {self.id} run ending due to SIM_TIME reached in state {self.state}")
                break

            current_state_processed_in_loop = False
            if self.state == "Home":
                yield self.env.process(self.at_home())
                current_state_processed_in_loop = True
            elif self.state == "General Practitioner":
                with self.resources["General Practitioner"].request() as req:
                    yield req
                    yield self.env.process(self.at_gp())
                current_state_processed_in_loop = True
            elif self.state in ["Geriatric Rehabilitation", "Short-Term Residential Care", "Crisis Admission"]:
                with self.resources[self.state].request() as req:
                    yield req
                    yield self.env.process(self.in_short_term_care())
                current_state_processed_in_loop = True
            elif self.state == "Awaiting Nursing Home (W2)":
                yield self.env.process(self.await_nursing_home_w2())
                current_state_processed_in_loop = True
            elif self.state == "Clinical Ward":
                with self.resources["Clinical Ward"].request() as req:
                    yield req
                    yield self.env.process(self.in_clinical_ward())
                current_state_processed_in_loop = True
            elif self.state == "Acute Admission Unit":
                with self.resources["Acute Admission Unit"].request() as req:
                    yield req
                    yield self.env.process(self.in_acute_admission())
                current_state_processed_in_loop = True
            elif self.state == "Emergency Department":
                with self.resources["Emergency Department"].request() as req:
                    yield req
                    yield self.env.process(self.in_emergency_dept())
                current_state_processed_in_loop = True
            elif self.state == "Nursing Home":
                with self.resources["Nursing Home"].request() as req:
                    yield req
                    yield self.env.process(self.in_nursing_home())
                current_state_processed_in_loop = True
            
            if not current_state_processed_in_loop and self.alive and self.state not in ["Discharge", "Passed Away"]:
                print(f"CRITICAL ERROR: Client {self.id} in unhandled state '{self.state}' at {self.env.now:.2f}. Stopping client.")
                self.alive = False # Prevent infinite loop for this client
                break


    def update_state(self, new_state):
        if not self.alive and self.state == "Passed Away" and new_state == "Passed Away": # Already processed
             return
        if not self.alive and new_state != "Passed Away":
            print(f"Warning: Client {self.id} is not alive (state: {self.state}) but tried to move to {new_state}")
            return

        duration = self.env.now - self.state_entry_time
        self.stats.record_time(self.state, duration)
        self.stats.record_transition(self.state, new_state)
        
        old_state = self.state
        self.state = new_state
        self.state_entry_time = self.env.now
        
        if self.state == "Passed Away":
            if self.alive: # Ensure it's only set once
                self.alive = False
        else:
            self.stats.record_visit(self.state)

        print(f"Client {self.id} moved from {old_state} to {self.state} at time {self.env.now:.2f} (Duration in {old_state}: {duration:.2f})")

    def _check_pass_away(self, probability, context_state):
        if self.alive and random.random() < probability:
            print(f"Client {self.id} passed away during {context_state} at time {self.env.now:.2f}")
            self.stats.record_pass_away()
            self.update_state("Passed Away")
            return True
        return False

    def at_home(self):
        if not self.alive: return
        if random.random() < EMERGENCY_PROB:
            self.update_state("Emergency Department")
        else:
            w1_wait_time = random.expovariate(1.0 / W1_MEAN)
            print(f"Client {self.id} entering W1 wait (for GP) for {w1_wait_time:.2f}h at {self.env.now:.2f}")
            yield self.env.timeout(w1_wait_time) # Wait for the W1 duration
            if not self.alive: return # Check if passed away by other means (if model gets complex)

            # Check for passing away ONLY at W1, after the wait
            if self._check_pass_away(PASS_AWAY_PROB, "W1 (waiting for GP)"):
                return 
            if not self.alive: return # Redundant if _check_pass_away is the only way to become not alive here

            print(f"Client {self.id} finished W1 wait, attempting to see GP at {self.env.now:.2f}")
            self.update_state("General Practitioner")

    def at_gp(self):
        if not self.alive: return
        gp_consult_time = random.uniform(0.25, 1)
        yield self.env.timeout(gp_consult_time)
        if not self.alive: return

        rand_stc = random.random()
        next_stc_type = ""
        if rand_stc < 0.3: next_stc_type = "Geriatric Rehabilitation"
        elif rand_stc < 0.6: next_stc_type = "Short-Term Residential Care"
        else: next_stc_type = "Crisis Admission"
        
        stc_admission_wait = random.uniform(0, 48)
        print(f"Client {self.id} from GP referred to {next_stc_type}, admission wait {stc_admission_wait:.2f}h at {self.env.now:.2f}")
        yield self.env.timeout(stc_admission_wait)
        if not self.alive: return
        self.update_state(next_stc_type)

    def in_short_term_care(self):
        if not self.alive: return
        current_stc_type = self.state
        stay_time = random.uniform(24, 72)
        yield self.env.timeout(stay_time)
        if not self.alive: return

        if random.random() < TRANSFER_PROB:
            options = ["Geriatric Rehabilitation", "Short-Term Residential Care", "Crisis Admission"]
            if current_stc_type in options: options.remove(current_stc_type) # Ensure it's removed
            if not options: # Should not happen if there are always other STC types
                 print(f"Warning: Client {self.id} in STC {current_stc_type}, no other STC options to transfer.")
            else:
                new_stc_type = random.choice(options)
                transfer_delay = random.uniform(6, 12)
                print(f"Client {self.id} transferring from {current_stc_type} to {new_stc_type} (delay {transfer_delay:.2f}h) at {self.env.now:.2f}")
                yield self.env.timeout(transfer_delay)
                if not self.alive: return
                self.update_state(new_stc_type)
                return 

        rand_next = random.random()
        next_state_after_stc = ""
        if rand_next < 0.4: next_state_after_stc = "Clinical Ward"
        elif rand_next < 0.7: next_state_after_stc = "Acute Admission Unit"
        else: next_state_after_stc = "Emergency Department"
        self.update_state(next_state_after_stc)

    def _handle_post_acute_care_transition(self): # This is a helper, not a SimPy process method
        if not self.alive: return
        self.update_state("Awaiting Nursing Home (W2)")

    def in_clinical_ward(self):
        if not self.alive: return
        stay_time = random.uniform(48, 120)
        yield self.env.timeout(stay_time)
        if not self.alive: return
        self._handle_post_acute_care_transition()

    def in_acute_admission(self):
        if not self.alive: return
        stay_time = random.uniform(24, 72)
        yield self.env.timeout(stay_time)
        if not self.alive: return
        self._handle_post_acute_care_transition()

    def in_emergency_dept(self):
        if not self.alive: return
        stay_time = random.uniform(6, 24)
        yield self.env.timeout(stay_time)
        if not self.alive: return
        self._handle_post_acute_care_transition()

    def await_nursing_home_w2(self):
        if not self.alive: return
        w2_wait_time = random.expovariate(1.0 / W2_MEAN)
        print(f"Client {self.id} in W2 (awaiting Nursing Home), wait: {w2_wait_time:.2f}h at {self.env.now:.2f}")
        yield self.env.timeout(w2_wait_time)
        if not self.alive: return
        self.update_state("Nursing Home")

    def in_nursing_home(self):
        if not self.alive: return
        stay_duration = random.uniform(NURSING_HOME_STAY_MIN_HOURS, NURSING_HOME_STAY_MAX_HOURS)
        print(f"Client {self.id} in Nursing Home, planned stay: {stay_duration:.2f}h at {self.env.now:.2f}")
        yield self.env.timeout(stay_duration)
        if not self.alive: return
        
        if random.random() < 0.5:
            print(f"Client {self.id} to be discharged (regular trajectory) from Nursing Home at {self.env.now:.2f}")
        else:
            print(f"Client {self.id} to be discharged (priority trajectory) from Nursing Home at {self.env.now:.2f}")
        self.update_state("Discharge")

def run_simulation():
    print("--- Initializing SimPy Environment ---") # Added print
    env = simpy.Environment()
    resources = {state: simpy.Resource(env, capacity=cap) for state, cap in CAPACITY.items()}
    stats = Stats(env, resources)

    clients = []
    print(f"--- Creating {NUM_CLIENTS} clients ---") # Added print
    for i in range(NUM_CLIENTS):
        client = Client(env, i, stats, resources)
        clients.append(client)
        env.process(client.run())
        # Removed conditional yield here to ensure run_simulation is a regular function

    print(f"--- Client creation complete. Starting simulation run until SIM_TIME: {SIM_TIME} ---") # Added print
    env.run(until=SIM_TIME)

    print(f"\n--- Simulation ended at time {env.now:.2f} ---")
    active_clients = 0
    for client in clients:
        if client.alive and client.state not in ["Discharge", "Passed Away"]:
            active_clients +=1
            stats.record_time(client.state, env.now - client.state_entry_time)
            stats.record_final_state(client.state)
        else: 
            stats.record_final_state(client.state) # Record final state for discharged/passed away too
    if NUM_CLIENTS > 0 : # Avoid printing if no clients were made (e.g. NUM_CLIENTS = 0)
        print(f"{active_clients} out of {NUM_CLIENTS} clients were still active (not Discharged or Passed Away) at SIM_TIME.")
    
    stats.print_summary()

if __name__ == "__main__":
    random.seed(42)
    run_simulation() # Calling run_simulation as a regular function now

--- Initializing SimPy Environment ---
--- Creating 100 clients ---
Client 0 starts at Home at time 0.00
Client 1 starts at Home at time 0.00
Client 2 starts at Home at time 0.00
Client 3 starts at Home at time 0.00
Client 4 starts at Home at time 0.00
Client 5 starts at Home at time 0.00
Client 6 starts at Home at time 0.00
Client 7 starts at Home at time 0.00
Client 8 starts at Home at time 0.00
Client 9 starts at Home at time 0.00
Client 10 starts at Home at time 0.00
Client 11 starts at Home at time 0.00
Client 12 starts at Home at time 0.00
Client 13 starts at Home at time 0.00
Client 14 starts at Home at time 0.00
Client 15 starts at Home at time 0.00
Client 16 starts at Home at time 0.00
Client 17 starts at Home at time 0.00
Client 18 starts at Home at time 0.00
Client 19 starts at Home at time 0.00
Client 20 starts at Home at time 0.00
Client 21 starts at Home at time 0.00
Client 22 starts at Home at time 0.00
Client 23 starts at Home at time 0.00
Client 24 starts at Home at ti

In [ ]:
import firebase_admin
from firebase_admin import credentials, firestore
import webbrowser
from datetime import datetime, timedelta
import pandas as pd
import folium
import colorsys

# 1) Initialize Firebase
cred = credentials.Certificate('healthy-healthcare-firebase-adminsdk-fbsvc-05315e3efc.json')
if not firebase_admin._apps:
    firebase_admin.initialize_app(cred)
db = firestore.client()

# 2) Helper: map performance ratio to semi-transparent RGBA color (green->red)
def perf_color(ratio, alpha=0.6):
    ratio = max(0, min(1, ratio))
    # Hue from red (0°) to green (120°)
    h = (120 * ratio) / 360
    r, g, b = colorsys.hls_to_rgb(h, 0.5, 1)
    return f'rgba({int(r*255)}, {int(g*255)}, {int(b*255)}, {alpha})'

# 3) CSS for dashboard: centered layout, rectangular tiles, side-by-side regions
box_css = '''
body {
  margin: 0;
  font-family: sans-serif;
  display: flex;
  flex-direction: column;
  align-items: center;
}
.regions-container {
  display: flex;
  flex-direction: row;
  justify-content: center;
  gap: 150px;
  flex-wrap: wrap;
}
.region {
  width: 380px;
  margin: 20px;
}
.region h2 {
  text-align: center;
  margin: 0 0 10px;
  font-size: 2em;
  font-weight: 700;
}
.grid {
  display: grid;
  grid-template-columns: repeat(3, 140px);
  grid-template-rows: repeat(3, 100px);
  grid-auto-flow: column;
  justify-content: center;
  gap: 10px;
}
.tile {
  width: 140px;
  height: 100px;
  display: flex;
  flex-direction: column;
  justify-content: center;
  align-items: center;
  border: 2px solid #000;
  border-radius: 8px;
  padding: 4px;
  color: #000;
}
.tile h3 {
  margin: 0 0 4px;
  font-size: 0.8em;
  font-weight: 600;
}
.tile p {
  margin: 0;
  font-size: 1em;
  font-weight: 700;
}
.button-container {
  text-align: center;
  margin: 30px 0;
}
.button-container button {
  padding: 12px 24px;
  font-size: 1.2em;
  border: none;
  background-color: #333;
  color: #fff;
  border-radius: 4px;
  cursor: pointer;
}
.button-container button:hover {
  background-color: #555;
}
'''

# 4) Build dashboard HTML
today = datetime.now()
base_date = datetime(2025, 5, 1)
week = 1
doc_key = (base_date + timedelta(weeks=week)).strftime('%Y-%m-%d')
municipalities = ['Arnhem', 'Nijmegen']

# Start HTML
dashboard_html = f"""<!DOCTYPE html>
<html>
<head>
  <style>{box_css}</style>
</head>
<body>
  <div class='regions-container'>
"""

# Gather max wait to normalize
waits = []
for city in municipalities:
    doc = db.collection('prediction').document(city).collection('weeks').document(f'week_{doc_key}').get()
    m = doc.to_dict().get('metrics', {})
    waits += [m.get('WAIT_W2', 0), m.get('WAIT_ELV', 0), m.get('WAIT_CRISIS', 0)]
max_wait = max(waits) if waits else 1

# Render each region side by side
for city in municipalities:
    doc = db.collection('prediction').document(city).collection('weeks').document(f'week_{doc_key}').get()
    m = doc.to_dict().get('metrics', {})
    # compute metrics
    nh_beds = m.get('NHBEDS', 0)
    nh_occ = m.get('OCC_NH', 0)
    nh_wait = m.get('WAIT_W2', 0)
    nh_avail = nh_beds * (1 - nh_occ)
    elv_beds = m.get('ELVBEDS', 0)
    elv_occ = m.get('OCC_ELV', 0)
    elv_wait = m.get('WAIT_ELV', 0)
    elv_avail = elv_beds * (1 - elv_occ)
    cr_beds = m.get('CRISISBEDS', 0)
    cr_occ = m.get('OCC_CRISIS', 0)
    cr_wait = m.get('WAIT_CRISIS', 0)
    cr_avail = cr_beds * (1 - cr_occ)
    # performance ratios
    ratios = {
        'nh_avail': nh_avail/nh_beds if nh_beds else 0,
        'nh_occ': 1-nh_occ,
        'nh_wait': 1-(nh_wait/max_wait),
        'elv_avail': elv_avail/elv_beds if elv_beds else 0,
        'elv_occ': 1-elv_occ,
        'elv_wait': 1-(elv_wait/max_wait),
        'cr_avail': cr_avail/cr_beds if cr_beds else 0,
        'cr_occ': 1-cr_occ,
        'cr_wait': 1-(cr_wait/max_wait)
    }
    dashboard_html += f"""
    <div class='region'>
      <h2>{city}</h2>
      <div class='grid'>
        <div class='tile' style='background:{perf_color(ratios['nh_avail'])}'>
          <h3>NH Available beds</h3><p>{int(nh_avail)}/{nh_beds}</p>
        </div>
        <div class='tile' style='background:{perf_color(ratios['nh_occ'])}'>
          <h3>NH Occupcancy Rate</h3><p>{nh_occ*100:.1f}%</p>
        </div>
        <div class='tile' style='background:{perf_color(ratios['nh_wait'])}'>
          <h3>NH Waiting time</h3><p>{nh_wait:.1f}d</p>
        </div>
        <div class='tile' style='background:{perf_color(ratios['elv_avail'])}'>
          <h3>ELV Available beds</h3><p>{int(elv_avail)}/{elv_beds}</p>
        </div>
        <div class='tile' style='background:{perf_color(ratios['elv_occ'])}'>
          <h3>ELV Occupancy rate</h3><p>{elv_occ*100:.1f}%</p>
        </div>
        <div class='tile' style='background:{perf_color(ratios['elv_wait'])}'>
          <h3>ELV Waiting time</h3><p>{elv_wait:.1f}d</p>
        </div>
        <div class='tile' style='background:{perf_color(ratios['cr_avail'])}'>
          <h3>Crisis Available beds</h3><p>{int(cr_avail)}/{cr_beds}</p>
        </div>
        <div class='tile' style='background:{perf_color(ratios['cr_occ'])}'>
          <h3>Crisis Occupancy rate</h3><p>{cr_occ*100:.1f}%</p>
        </div>
        <div class='tile' style='background:{perf_color(ratios['cr_wait'])}'>
          <h3>Crisis Waiting time</h3><p>{cr_wait:.1f}d</p>
        </div>
      </div>
    </div>
"""
# Close regions container and add button

dashboard_html += f"""
  </div>  <!-- .regions-container -->
  <div class='button-container'>
    <button onclick="window.location.href='map.html'">View Map of Facilities</button>
  </div>
</body>
</html>
"""

with open('dashboard.html', 'w') as f:
    f.write(dashboard_html)
webbrowser.open('dashboard.html')

# 5) Map code unchanged follows her

# ─── 4) (mapping code unchanged) ─────────────────────────────────────
# ... your existing map-building logic follows here ..
map_rows = []
for city in municipalities:
    doc = db.collection('prediction')\
             .document(city)\
             .collection('weeks')\
             .document(f'week_{doc_key}').get()
    data = doc.to_dict() if doc.exists else {"active_patients_data":[], "metrics":{}}
    m = data.get('metrics', {})
    map_rows.append({
        'place_name':    city,
        'type':          'Municipality',
        'municipality':  city,
        'lat':           municipality_info[city]['lat'],
        'lon':           municipality_info[city]['lon'],
        'beds':          m.get('NHBEDS',0),
        'active_total':  len(data.get('active_patients_data',[])),
        'occupancy_tot': m.get('OCC_NH',0),
        'wait_tot':      m.get('WAIT_W2',0)
    })

# 4b) load nursing homes from CSVs
df_arn = pd.read_csv('arnhem.csv', dtype={'Postcode': str}); df_arn['municipality']='Arnhem'
df_nij = pd.read_csv('nijmegen.csv', dtype={'Postcode': str}); df_nij['municipality']='Nijmegen'
df_nh  = pd.concat([df_arn, df_nij], ignore_index=True)
# normalize to 6-char code
df_nh['Postcode_norm'] = (
    df_nh['Postcode'].str.upper()
           .str.replace(r"\s+","",regex=True)
 )

# 4c) load or cache full Dutch postcode CSV locally to speed future runs
pc_url = 'https://raw.githubusercontent.com/fatahfattah/6pp/main/6pp.csv'
cache_path = pathlib.Path('6pp.csv')
if not cache_path.exists():
    df_pc_full = pd.read_csv(pc_url, usecols=['postcode','latitude','longitude'], dtype=str)
    df_pc_full.to_csv(cache_path, index=False)
else:
    df_pc_full = pd.read_csv(cache_path, usecols=['postcode','latitude','longitude'], dtype=str)
# normalize postal codes in full dataset
df_pc_full['postcode_norm'] = (
    df_pc_full['postcode'].str.upper()
               .str.replace(r"\s+","",regex=True)
 )
postcode_map = {
    r['postcode_norm']: (float(r['latitude']), float(r['longitude']))
    for _,r in df_pc_full.iterrows()
}

# 4d) assign coords for nursing homes
def get_coords(pc, town):
    coords = postcode_map.get(pc)
    if coords:
        return coords
    return (municipality_info[town]['lat'], municipality_info[town]['lon'])

for _, row in df_nh.iterrows():
    lat, lon = get_coords(row['Postcode_norm'], row['municipality'])
    map_rows.append({
        'place_name':    row['Naam'],
        'type':          'Nursing Home',
        'municipality':  row['municipality'],
        'lat':           lat,
        'lon':           lon,
        'beds':          row.get('Aantal Plaatsen Totaal',0),
        'active_total':  None,
        'occupancy_tot': None,
        'wait_tot':      None
    })

# 4e) pro-rata distribution
df_map = pd.DataFrame(map_rows)
beds_by_city = df_map.groupby('municipality')['beds'].sum()
metrics = {}
for city in municipalities:
    base = df_map.query("municipality==@city and type=='Municipality'").iloc[0]
    metrics[city] = {
        'active':    base['active_total'],
        'occupancy': base['occupancy_tot'],
        'wait':      base['wait_tot']
    }
def dist_active(r):
    if r['type']=='Municipality': return metrics[r['municipality']]['active']
    return round(metrics[r['municipality']]['active'] * (r['beds']/beds_by_city[r['municipality']]))

df_map['active_patients'] = df_map.apply(dist_active, axis=1)
df_map['occupancy_rate']   = df_map['municipality'].map(lambda c: metrics[c]['occupancy'])
df_map['wait_days']        = df_map['municipality'].map(lambda c: metrics[c]['wait'])

# 4f) render folium map
g = folium.Map(location=[51.85, 5.9], zoom_start=10)
def marker_color(t): return 'red' if t == 'Municipality' else 'blue'

for _, r in df_map.iterrows():
    available_beds = int(r['beds'] * (1 - r['occupancy_rate']))
    popup_html = (
        f"<b>{r['place_name']}</b><br>"
        f"Available Beds: {available_beds}<br>"
        f"Active Patients: {r['active_patients']}<br>"
        f"Occupancy Rate: {r['occupancy_rate']*100:.1f}%<br>"
        f"Waiting Time: {r['wait_days']:.1f} days"
    )
    folium.Marker(
        [r['lat'], r['lon']],
        tooltip=r['place_name'],
        popup=folium.Popup(popup_html, max_width=300),
        icon=folium.Icon(color=marker_color(r['type']))
    ).add_to(g)

g.save('map.html')